In [1]:
%load_ext autoreload
%autoreload 2
import sys
# sys.path.append(r'E:/0code')
sys.path.append(r'/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/0code/mylearn')
sys.path.append(r'/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/0code')
# sys.path.append('/Users/yanbin/Documents/Projects/AI-Middle-Project/')
# sys.path.append('/Users/yanbin/Documents/Projects/mylearn')

# sys.path.append('/home/wyf/0code/AI-Middle-Project/')
# sys.path.append('/home/wyf/0code/mylearn')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from logger import get_logger
from neural_network import MLPClassifier
from pyml.model_selection import KFold
from pyml.model_selection import ShuffleSplit
from pyml.metrics.classification import precision_score


mylogger = get_logger(__name__)
mylogger.debug('hello world')

[DEBUG] <ipython-input-1-8dd56ce29e25> <ipython-input-1-8dd56ce29e25> - <module> line 23 - 
hello world
 - (2018-10-21 02:31:06)

[DEBUG] __init__ __init__.py - pylab_setup line 90 - 
backend module://ipykernel.pylab.backend_inline version unknown
 - (2018-10-21 02:31:06)



# 读取数据

In [2]:
train_X_vec2doc_2 = np.load('../data/wyf-train_X-doc2vec-500.npy')
test_X_vec2doc_2 = np.load('../data/wyf-test_X-doc2vec-500.npy')

In [3]:
train_X_vec2doc = np.load('../data/clean/trainX_vec2doc-24000-500.npy')
test_X_vec2doc = np.load('../data/clean/testX_vec2doc-6000-500.npy')

In [4]:
train_X_tfidf = np.load('../data/clean/trainX_tfidf-24000-62761.npy')
test_X_tfidf = np.load('../data/clean/testX_tfidf-6000-62761.npy')

In [5]:
train_X_onehot = np.load('../data/clean/trainX_onehot-24000-62761.npy')
test_X_onehot = np.load('../data/clean/testX_onthot-6000-62761.npy')

In [6]:
train_ori_Y = np.load('../data/clean/trainY_24000.npy')

# 使用one-hot进行交叉验证

In [7]:
train_X = train_X_onehot.T
test_X = test_X_onehot.T
train_Y  = train_ori_Y.reshape((-1,1))
train_X.shape,train_Y.shape,test_X.shape

((62761, 24000), (24000, 1), (62761, 6000))

In [8]:
train_X = pd.DataFrame(train_X)
test_X = pd.DataFrame(test_X)
train_Y = pd.DataFrame(train_Y)
train_X.shape,train_Y.shape,test_X.shape

((62761, 24000), (24000, 1), (62761, 6000))

In [ ]:
# learning_rates = [0.5, 0.3, 0.1, 0.05, 0.02, 0.01]

learning_rates = [1,0.5]

n_splits = 1
ms = ShuffleSplit(n_splits=n_splits,test_size=0.2)
models = []
for train_indices, valid_indices in ms.split(train_X_onehot):
    for learning_rate in learning_rates:
        train_train_X = train_X.iloc[:,train_indices]
        train_train_Y = train_Y.iloc[train_indices,:]
        train_valid_X = train_X.iloc[:,valid_indices]
        train_valid_Y = train_Y.iloc[valid_indices,:]
        mlpc = MLPClassifier(verbose=False, 
            # 忽略输入和输出层。中间层的结点数。例如 本例子为 input -> 8 -> 4 -> 2（2分类）
            hidden_layer_sizes=(4,), 
            activation='relu', # 'relu', 'sigmoid'. 更多函数，等待支持
            max_iter=1000, # 迭代次数
            learning_rate_init=learning_rate, 
            warm_start=True,  # True就好了
            mini_batch='auto', # 保持auto
            step_size=10, # 多少个iteration后，才进行“输出loss”，输出“准确度”，保存模型文件，这三个操作?
            load_from_file=False, # 读文件恢复模型吗
            dump_file=False, # 要不要把模型写入文件？
            validation_set=(train_valid_X, train_valid_Y) # 要不要给一个validation set？（仅用来输出这个集合的准确度)
        )
        mlpc.fit(train_train_X, train_train_Y)
        r = mlpc.score(train_valid_X, train_valid_Y)
        models.append(mlpc)
        print(r)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[0] loss 0.6977883883510524
 - (2018-10-21 02:34:20)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5129166666666667, training score 0.5057291666666667
 - (2018-10-21 02:34:23)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[10] loss 5.932949211450329
 - (2018-10-21 02:34:39)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.4870833333333333, training score 0.48489583333333336
 - (2018-10-21 02:34:42)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[20] loss 5.872959937320905
 - (2018-10-21 02:34:57)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.4870833333333333, training score 0.4901041666666667
 - (2018-10-21 02:35:00)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[30] loss 5.65

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.4870833333333333, training score 0.490625
 - (2018-10-21 02:42:00)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[280] loss 5.806971735778543
 - (2018-10-21 02:42:14)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.4870833333333333, training score 0.49583333333333335
 - (2018-10-21 02:42:17)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[290] loss 5.66299747786793
 - (2018-10-21 02:42:30)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.4870833333333333, training score 0.5083333333333333
 - (2018-10-21 02:42:33)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[300] loss 5.752981389062062
 - (2018-10-21 02:42:46)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.487

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[550] loss 5.716987824584409
 - (2018-10-21 02:49:51)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.4870833333333333, training score 0.5036458333333333
 - (2018-10-21 02:49:54)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[560] loss 5.824968518017369
 - (2018-10-21 02:50:09)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.4870833333333333, training score 0.4942708333333333
 - (2018-10-21 02:50:12)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[570] loss 5.579012494086738
 - (2018-10-21 02:50:27)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.4870833333333333, training score 0.515625
 - (2018-10-21 02:50:30)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[580] loss 5.734984606

In [ ]:
for i,model in enumerate(models):
    m = model.information
    num_iterations = list(m.keys())
    costs = [m[i][0]  for i in num_iterations]
    valid_scores = [m[i][1]  for i in num_iterations]
    train_scores = [m[i][2]  for i in num_iterations]
    plt.plot(num_iterations, valid_scores, '-', label='valid '+ str(learning_rates[i]))
#     plt.plot(num_iterations, train_scores, ':', label='train '+ str(learning_rates[i]))
    plt.legend()

## 使用one-hot进行交叉验证

In [61]:
train_X = train_X_onehot.T
test_X = test_X_onehot.T
train_Y  = train_ori_Y.reshape((-1,1))
train_X.shape,train_Y.shape,test_X.shape

((62761, 24000), (24000, 1), (62761, 6000))

In [62]:
train_X = pd.DataFrame(train_X)
test_X = pd.DataFrame(test_X)
train_Y = pd.DataFrame(train_Y)
train_X.shape,train_Y.shape,test_X.shape

((62761, 24000), (24000, 1), (62761, 6000))

In [63]:
learning_rates = [0.5, 0.3, 0.1, 0.05, 0.02, 0.01]

n_splits = 1
ms = ShuffleSplit(n_splits=n_splits,test_size=0.2)
models = []
for train_indices, valid_indices in ms.split(train_X):
    for learning_rate in learning_rates:
        train_train_X = train_X.iloc[:,train_indices]
        train_train_Y = train_Y.iloc[train_indices,:]
        train_valid_X = train_X.iloc[:,valid_indices]
        train_valid_Y = train_Y.iloc[valid_indices,:]
        mlpc = MLPClassifier(verbose=False, 
            # 忽略输入和输出层。中间层的结点数。例如 本例子为 input -> 8 -> 4 -> 2（2分类）
            hidden_layer_sizes=(4,), 
            activation='relu', # 'relu', 'sigmoid'. 更多函数，等待支持
            max_iter=5000, # 迭代次数
            learning_rate_init=learning_rate, 
            warm_start=True,  # True就好了
            mini_batch='auto', # 保持auto
            step_size=10, # 多少个iteration后，才进行“输出loss”，输出“准确度”，保存模型文件，这三个操作?
            load_from_file=False, # 读文件恢复模型吗
            dump_file=False, # 要不要把模型写入文件？
            validation_set=(train_valid_X, train_valid_Y) # 要不要给一个validation set？（仅用来输出这个集合的准确度)
        )
        mlpc.fit(train_train_X, train_train_Y)
        r = mlpc.score(train_valid_X, train_valid_Y)
        models.append(mlpc)
        print(r)

IndexError: positional indexers are out-of-bounds

In [5]:
for i,model in enumerate(models):
    m = model.information
    num_iterations = list(m.keys())
    costs = [m[i][0]  for i in num_iterations]
    valid_scores = [m[i][1]  for i in num_iterations]
    train_scores = [m[i][2]  for i in num_iterations]
    plt.plot(num_iterations, valid_scores, '-', label='valid '+ str(learning_rates[i]))
#     plt.plot(num_iterations, train_scores, ':', label='train '+ str(learning_rates[i]))
    plt.legend()

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[0] loss 0.696822080402499
 - (2018-10-21 01:43:42)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5125, training score 0.5333333333333333
 - (2018-10-21 01:43:42)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[1] loss 0.6885862587826703
 - (2018-10-21 01:43:42)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5125, training score 0.55
 - (2018-10-21 01:43:42)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[2] loss 0.7138386168290861
 - (2018-10-21 01:43:42)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.4875, training score 0.5666666666666667
 - (2018-10-21 01:43:42)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[3] loss 0.696635905477929
 - (2018-10-21 01:43:42)

[INFO] multil

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[29] loss 0.6926466556173787
 - (2018-10-21 01:43:43)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5125, training score 0.6
 - (2018-10-21 01:43:44)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[30] loss 0.6855517888748716
 - (2018-10-21 01:43:44)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5125, training score 0.55
 - (2018-10-21 01:43:44)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[31] loss 0.6775419659775401
 - (2018-10-21 01:43:44)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5125, training score 0.6
 - (2018-10-21 01:43:44)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[32] loss 0.6780218095361439
 - (2018-10-21 01:43:44)

[INFO] multilayer_perceptron multilay

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[58] loss 0.6985909497520317
 - (2018-10-21 01:43:45)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5525, training score 0.5
 - (2018-10-21 01:43:45)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[59] loss 0.6813474685688689
 - (2018-10-21 01:43:45)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.545, training score 0.6333333333333333
 - (2018-10-21 01:43:45)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[60] loss 0.6833330838394475
 - (2018-10-21 01:43:45)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.5125, training score 0.6333333333333333
 - (2018-10-21 01:43:45)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[61] loss 0.6968571746427905
 - (2018-10-21 01:43:45)

[INFO] mu

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[87] loss 0.6319009055798794
 - (2018-10-21 01:43:46)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.64, training score 0.7666666666666667
 - (2018-10-21 01:43:46)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[88] loss 0.6802273001169982
 - (2018-10-21 01:43:46)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.57, training score 0.6
 - (2018-10-21 01:43:46)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[89] loss 0.6778441364222075
 - (2018-10-21 01:43:46)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.7325, training score 0.6666666666666666
 - (2018-10-21 01:43:46)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[90] loss 0.6411405306092387
 - (2018-10-21 01:43:46)

[INFO] multi

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.6975, training score 0.6333333333333333
 - (2018-10-21 01:43:48)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[116] loss 0.5544448139586485
 - (2018-10-21 01:43:48)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.705, training score 0.75
 - (2018-10-21 01:43:48)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[117] loss 0.5414694444769484
 - (2018-10-21 01:43:48)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.6575, training score 0.7166666666666667
 - (2018-10-21 01:43:48)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[118] loss 0.4773877036468433
 - (2018-10-21 01:43:48)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.6525, training score 0.7333333333333333
 

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[144] loss 0.3495176365171011
 - (2018-10-21 01:43:49)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.7175, training score 0.9
 - (2018-10-21 01:43:49)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[145] loss 0.4909301436659727
 - (2018-10-21 01:43:49)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.725, training score 0.7666666666666667
 - (2018-10-21 01:43:49)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[146] loss 0.5885938038945885
 - (2018-10-21 01:43:49)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.715, training score 0.7166666666666667
 - (2018-10-21 01:43:49)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[147] loss 0.5006778429538004
 - (2018-10-21 01:43:49)

[INFO]

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.835, training score 0.8166666666666667
 - (2018-10-21 01:43:50)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[173] loss 0.322356859418609
 - (2018-10-21 01:43:50)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.865, training score 0.9333333333333333
 - (2018-10-21 01:43:51)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[174] loss 0.3606575835707872
 - (2018-10-21 01:43:51)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.8575, training score 0.9166666666666666
 - (2018-10-21 01:43:51)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[175] loss 0.24673562753038875
 - (2018-10-21 01:43:51)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.8925, training score 0.96666

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[201] loss 0.19658662118700926
 - (2018-10-21 01:43:52)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.92, training score 0.95
 - (2018-10-21 01:43:52)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[202] loss 0.1749832247816049
 - (2018-10-21 01:43:52)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.9, training score 0.95
 - (2018-10-21 01:43:52)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[203] loss 0.19473500861547546
 - (2018-10-21 01:43:52)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 225 - 
validation score 0.9225, training score 0.9166666666666666
 - (2018-10-21 01:43:52)

[INFO] multilayer_perceptron multilayer_perceptron.py - __train line 207 - 
[204] loss 0.1689939749503063
 - (2018-10-21 01:43:52)

[INFO] multilayer_pe

/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/0code/mylearn/neural_network/multilayer_perceptron.py:345: RuntimeWarning: divide by zero encountered in log
  trans = np.where(Y == i_node, -np.log(ai), -np.log(1-ai))
/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/0code/mylearn/neural_network/multilayer_perceptron.py:355: RuntimeWarning: divide by zero encountered in true_divide
  ret[row] = np.where(Y == row, -1/ai, 1/(1-ai))
/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/0code/mylearn/neural_network/multilayer_perceptron.py:277: RuntimeWarning: invalid value encountered in multiply
  epsilon = sm * der
/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/0code/mylearn/neural_network/multilayer_perceptron.py:280: RuntimeWarning: divide by zero encountered in true_divide
  mylogger.debug('bp(%s) left\n%s\nright\n%s', i_layer, -(Y/A), (1-Y)/(1-A))
/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/0code/mylearn/neural_network/multilayer_perceptron.py:280: RuntimeWarning: invalid value encountered in true_divide
  mylogger.debug('bp(%s) left\n%s\nright\n%s', i_la

AssertionError: 